# Cálcula posición de Asteroide 2024 YR4
## Por Jorge I. Zuluaga

In [12]:
!pip install -Uq pymcel rebound


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [13]:
!pip3 install --upgrade certifi


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [14]:
import pymcel as pc
import spiceypy as spy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rebound as rb
from astropy.time import Time
from tqdm import tqdm
from time import time

rad = 180/np.pi
deg = 1/rad
display.precision = 17
np.set_printoptions(precision=17)

Unidades:

In [15]:
au = pc.constantes.au
mu = pc.constantes.mu_sun
dia = 86400

## Información requerida

Kernels de SPICE:

In [16]:
pc.prepara_spice()

Descarga información sobre el asteroide:

In [17]:
from urllib import request
import json
def obtiene_elementos_asteroide(id, verbose=True):
    """Obtiene los elementos orbitales de un asteroide, y las covarianzas
    de sus elementos

    Ejemplo:
        >>> promedios,covarianza = obtiene_elementos_asteroide('2024yr4',verbose=True)

    Notas:
      - Basado en el código de Leonard Gómez, Astronomía UdeA (2022)
    """
    url = f"https://ssd-api.jpl.nasa.gov/sbdb.api?sstr={id}&cov=mat&full-prec=true"
    if verbose:
      print(f"Descargando los datos para {id.upper()} de {url}...")
    html=request.urlopen(url)
    json_data=json.loads(html.read().decode())

    t0=float(json_data["orbit"]["epoch"])
    if verbose:
      print(f"Epoca de los elementos (JDTDB): {t0}")

    if verbose:
      print(f"Extrayendo la matriz de covarianza...")
    Cov=np.array(json_data["orbit"]["covariance"]["data"],dtype=float)
    Cov_label=json_data["orbit"]["covariance"]["labels"]
    t=float(json_data["orbit"]["epoch"])

    if verbose:
      print(f"Extrayendo los elementos y sus errores...")
    nlen=len(json_data["orbit"]["elements"])
    elnames=[]
    elements=dict()
    for i in range(nlen):
      element=json_data["orbit"]["elements"][i]
      elements[element["name"]]=dict()
      for prop in element.keys():
        try:
          elements[element["name"]][prop]=float(element[prop])
        except:
          pass

    for elname in elements.keys():
      element=elements[elname]
      print(f"Elemento {elname} = {element['value']:.7e} +/- {element['sigma']:.7e}")

    means=[elements['e']['value'],elements['q']['value'],elements['tp']['value'],elements['om']['value'],elements['w']['value'],elements['i']['value']]
    if verbose:
      print(f"Orden de elementos orbitales: {Cov_label}")

    return t0,means,Cov

In [18]:
obtiene_elementos_asteroide('2024YR4')

Descargando los datos para 2024YR4 de https://ssd-api.jpl.nasa.gov/sbdb.api?sstr=2024YR4&cov=mat&full-prec=true...


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

In [8]:
jd_0,promedios,covarianza = pc.obtiene_elementos_asteroide(id='2024YR4',verbose=True)

Descargando los datos para 2024YR4 de https://ssd-api.jpl.nasa.gov/sbdb.api?sstr=2024YR4&cov=mat&full-prec=true...


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

## Cálculo de posición inicial

Extrae elementos orbitales promedios:

In [ ]:
# Elementos orbitales en unidades
e=promedios[0]
q=promedios[1]*au
jd_p=promedios[2]
node=promedios[3]*deg
peri=promedios[4]*deg
i=promedios[5]*deg

# Elementos derivados
a = q/(1-e)
n = np.sqrt(mu/a**3)
M_0 = np.mod(n*(jd_0 - jd_p)*dia,2*np.pi)
et_0 = spy.unitim(jd_0,'JDTDB','ET')

e, q/au, jd_p, node*rad, peri*rad, i*rad, M_0*rad, n*dia/deg

(0.6616262862811932,
 0.8515141727406684,
 2460636.9247331256,
 271.36819100044517,
 134.3640682810488,
 3.40836200033265,
 13.721220273589012,
 0.24689436588042127)

Calculo del vector de estado:

In [ ]:
X_ast_0 = spy.conics([q,e,i,node,peri,M_0,0,mu],0)
r_ast_0 = X_ast_0[:3]
v_ast_0 = X_ast_0[3:]
r_ast_0, v_ast_0

(array([-7.9987254970992462e+10,  1.5555243416642734e+11,
        -4.5412701563402786e+09]),
 array([-34063.31203750673   ,     99.28749792575036,
         -2028.0029874890838 ]))

Comparando con la consulta en Horizons:

In [ ]:
tabla, tiempos, X_ast_hor_0 = pc.consulta_horizons('2024 YR4','@10',epochs=jd_0,
                                                   propiedades=[
                                                       ('x','m'),('y','m'),('z','m'),
                                                       ('vx','m/s'),('vy','m/s'),('vz','m/s')
                                                   ])
r_ast_hor_0 = X_ast_hor_0[:3]
v_ast_hor_0 = X_ast_hor_0[3:]
r_ast_hor_0, v_ast_hor_0

(array([-7.9987254969828949e+10,  1.5555243416642398e+11,
        -4.5412701562710180e+09]),
 array([-34063.312037574404  ,     99.28749805739092,
         -2028.0029874929282 ]))

La diferencia entre ambas predicciones son:

In [ ]:
np.linalg.norm(r_ast_hor_0 - r_ast_0), np.linalg.norm(v_ast_0 - v_ast_hor_0) # metros

(1.1655776369372117, 1.4806668878019845e-07)

Una diferencia de cerca de 1 metro

## Predicción posición futura

### Usando la teoría del problema de los dos cuerpos

In [ ]:
from astropy.time import Time

Fecha de impacto de acuerdo a [NASA Sentry](https://cneos.jpl.nasa.gov/sentry/details.html#?des=2024%20YR4):

In [ ]:
fecha = '2032-12-22.59'
dias = 22.59
dia_int = int(dias)
hora = (dias - dia_int)*24
hh = int(hora)
minuto = (hora - hh)*60
mm = int(minuto)
ss = (minuto - mm)*60
f"{hh}:{mm}:{ss}"

'14:9:35.99999999998772'

Cálculo de día juliano:

In [ ]:
fecha_imp = f"2032-12-22 {hh}:{mm}:{ss:.1f} UTC"
et_imp = spy.utc2et(fecha_imp)
dt_imp = spy.deltet(et_imp,'ET')
et_imp = et_imp - dt_imp
jd_imp = spy.unitim(et_imp,'ET','JDTDB')
fecha_imp, jd_imp

('2032-12-22 14:9:36.0 UTC', 2463589.09)

Calculemos el vector de estado usando el problema de los dos cuerpos:

In [ ]:
X_ast_imp = spy.conics([q,e,i,node,peri,M_0,et_0,mu],et_imp)
r_ast_imp = X_ast_imp[:3]
r_ast_imp

array([-1.9739494249543694e+10,  1.4892979910970111e+11,
       -9.6351049155072832e+08])

Comparemos con la preducción de Horizons:

In [ ]:
tabla, tiempos, r_ast_hor_imp = pc.consulta_horizons('2024 YR4','@10',epochs=jd_imp,
                                                     propiedades=[('x','m'),('y','m'),('z','m')])
r_ast_hor_imp

array([-2.2783160694547644e+09,  1.4718742789457751e+11,
       -1.8970705739349134e+07])

Cuál es la diferencia:

In [ ]:
error_ast_imp = np.linalg.norm(r_ast_hor_imp - r_ast_imp)
error_ast_imp/1e3/1e6

17.57329668261153

¡Se ve que la diferencia es enorme!

Calculemos ahora la distancia del Asteroide a la Tierra:

In [ ]:
tabla, ts, r_tierra_imp = pc.consulta_horizons('399','@10',epochs=jd_imp,
                                               propiedades=[('x','m'),('y','m'),('z','m')])
r_tierra_imp

array([-2.1175193833051431e+09,  1.4713355999800201e+11,
       -1.0349804319128396e+07])

In [ ]:
r_tierra_ast_imp = r_tierra_imp - r_ast_imp
d_tierra_ast = np.linalg.norm(r_tierra_ast_imp)
d_tierra_ast/1e3/1e6

17.738911703608668

Pero con Horizons:

In [ ]:
r_tierra_ast_imp = r_tierra_imp - r_ast_hor_imp
d_tierra_ast = np.linalg.norm(r_tierra_ast_imp)
d_tierra_ast/1e3/1e6

0.16979883538901488

Que es una distancia significativamente menor. Esto significa que necesitamos un model mejor. Probaremos ahora el problema de los n-cuerpos.

### Problema N-cuerpos

In [ ]:
sim = rb.Simulation()
sim.units = 'kg', 'm', 's'

todos_los_cuerpos = [
    'Sun',
    '199',
    '299',
    '399',
    '301',
    '499',
    '599',
    '699',
    '799',
    '899'
]

sistema = []
for n,cuerpo in enumerate(todos_los_cuerpos):
  sim.add(cuerpo,date=f"JD{jd_0}")
  sistema += [
      dict(m=sim.particles[n].m,r=sim.particles[n].xyz,v=sim.particles[n].vxyz)
  ]
sim.move_to_hel()
sim.particles[3].r = 6371e3

print("Verifying positions:")
for n,cuerpo in enumerate(todos_los_cuerpos):
  tabla,ts,r_cuerpo_hor_0 = pc.consulta_horizons(cuerpo,'@10',epochs=jd_0,propiedades=[('x','m'),('y','m'),('z','m')])
  print("\tSimulation: ",sim.particles[n].xyz)
  print("\tHorizons: ", r_cuerpo_hor_0)

sim.save_to_file('sistema-solar.bin')

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for '199'... 
Found: Mercury (199) 
Searching NASA Horizons for '299'... 
Found: Venus (299) 
Searching NASA Horizons for '399'... 
Found: Earth (399) 
Searching NASA Horizons for '301'... 
Found: Moon (301) 
Searching NASA Horizons for '499'... 
Found: Mars (499) 
Searching NASA Horizons for '599'... 
Found: Jupiter (599) 
Searching NASA Horizons for '699'... 
Found: Saturn (699) 
Searching NASA Horizons for '799'... 
Found: Uranus (799) 
Searching NASA Horizons for '899'... 
Found: Neptune (899) 
Verifying positions:
	Simulation:  [0.0, 0.0, 0.0]
	Horizons:  [0. 0. 0.]
	Simulation:  [-23558961349.561214, -65496721293.21561, -3191625291.3508234]
	Horizons:  [-2.3558961349561211e+10 -6.5496721293215622e+10 -3.1916252913508263e+09]
	Simulation:  [24538569682.375435, 104955180628.24815, 25521234.91796418]
	Horizons:  [2.4538569682375443e+10 1.0495518062824812e+11 2.5521234917961061e+07]
	Simulation:  [-6628758

Agregamos el asteroide:

In [ ]:
sim = rb.Simulation('sistema-solar.bin')
sim.add(m=0,
        x=r_ast_0[0],y=r_ast_0[1],z=r_ast_0[2],
        vx=v_ast_0[0],vy=v_ast_0[1],vz=v_ast_0[2])
sim.integrator = 'ias15'
sim.dt = 100

Integramos hasta el tiempo de impacto

In [ ]:
tiempo = jd_imp - jd_0
tiempo_seg = tiempo*dia
sim.integrate(tiempo_seg)
sim.move_to_hel()
table,ts,r_cuerpo_hor = pc.consulta_horizons('399','@10',epochs=jd_0+tiempo,propiedades=[('x','m'),('y','m'),('z','m')])

Calculemos la distancia entre la Tierra y el asteroide:

In [ ]:
np.linalg.norm(np.array(sim.particles[3].xyz) - np.array(sim.particles[-1].xyz))/1e3/1e6

0.167798156376097

## Rutina de integración

In [ ]:
def integra_sistema_solar_asteroide(elementos,jd):

  # Carga simulacion
  sim = rb.Simulation('sistema-solar.bin')
  sim.integrator = 'ias15'
  sim.dt = 100

  # Elementos: ['e', 'q', 'tp', 'node', 'peri', 'i']
  e = elementos[0]
  q = elementos[1]*au
  jd_p = elementos[2]
  node = elementos[3]*deg
  peri = elementos[4]*deg
  i = elementos[5]*deg

  # Elementos derivados
  a = q/(1-e)
  n = np.sqrt(mu/a**3)
  M_0 = np.mod(n*(jd_0 - jd_p)*dia,2*np.pi)

  # A partir de los elementos calcula la posición inicial del asteroide
  X_ast_0 = spy.conics([q,e,i,node,peri,M_0,0,mu],0)
  r_ast_0 = X_ast_0[:3]
  v_ast_0 = X_ast_0[3:]

  sim.add(m=0,
          x=r_ast_0[0],y=r_ast_0[1],z=r_ast_0[2],
          vx=v_ast_0[0],vy=v_ast_0[1],vz=v_ast_0[2])
  tiempo = jd - jd_0
  tiempo_seg = tiempo*dia
  sim.integrate(tiempo_seg)
  sim.move_to_hel()

  # Posición asteroide
  r_ast = np.array(sim.particles[-1].xyz)
  r_tierra = np.array(sim.particles[3].xyz)
  d = np.linalg.norm(r_ast - r_tierra)

  # Devuelve posición del asteroide y de la Tierra
  return r_ast/au, r_tierra/au, d/au


In [ ]:
r_ast_imp, r_tierra_imp, d_ast_tierra_imp = integra_sistema_solar_asteroide(promedios,jd_imp)
d_ast_tierra_imp

0.0011216613952520448

## Genera muchos asteroides y calcula impacto

In [ ]:
np.random.seed = 3
muestra_elementos = np.random.multivariate_normal(promedios,covarianza,100)

In [ ]:
resultados = []
for elementos in tqdm(muestra_elementos):
  r_ast_imp, r_tierra_imp, d_ast_tierra_imp = integra_sistema_solar_asteroide(elementos,jd_imp)
  resultados += [dict(
      x_ast=r_ast_imp[0],
      y_ast=r_ast_imp[1],
      z_ast=r_ast_imp[2],
      d_ast_tie=d_ast_tierra_imp
  )]
resultados = pd.DataFrame(resultados,columns=['x_ast','y_ast','z_ast','x_tie','y_tie','z_tie','d_ast_tie'])

100%|██████████| 100/100 [00:24<00:00,  4.15it/s]


In [ ]:
filtro = (resultados.d_ast_tie < 6371e3/au)
filtro.sum()

4

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
fig = go.Figure(
    data = [
        go.Scatter(
          x=resultados.x_ast,
          y=resultados.y_ast,
          name='Asteroid',
          mode='markers',
          marker=dict(size=3)
        ),
    ],
    layout = dict(
      width = 600,
      height = 600,
      template = 'plotly_dark',
      yaxis=dict(scaleanchor="x", scaleratio=1)
    )
)

R = 6371e3/au
fig.add_shape(
    type='circle',
    xref='x', yref='y',
    x0=r_tierra_imp[0] - R, y0=r_tierra_imp[1] - R,
    x1=r_tierra_imp[0] + R, y1=r_tierra_imp[1] + R,
    line_color='Green'
)

fig.show()

## Otra aproximación

In [ ]:
# Define funciones útiles
Nt = 50
it = 0
inicio = time()
paso = time()
def heartbeat(simulacion):
  global it,Nt,inicio,paso
  tiempo = jd_imp - jd_0
  tiempo_seg = tiempo*dia
  sim = simulacion.contents
  if ((sim.t - it*tiempo_seg/Nt)>0) or it == 0:
    fin = time()
    print(f"Tiempo: {sim.t/tiempo_seg:.2e} (it = {it}/{Nt}, tiempo = {fin-paso:.2f} seg, faltan = {(Nt-1-it)*(fin-paso):.2f} seg)")
    paso = time()
    it += 1

def integra_sistema_solar_asteroide_2(muestra_elementos,jd):
  sim = rb.Simulation('sistema-solar.bin')

  # Tiempo de integración
  tiempo = jd - jd_0
  tiempo_seg = tiempo*dia
  print(f"Integrando hasta: {tiempo_seg}")

  choques = []
  def que_hacer_en_colision(simulacion,colision):
    global choques
    sim = simulacion.contents
    xyz_1 = np.array(sim.particles[int(colision.p1)].xyz)
    xyz_2 = np.array(sim.particles[int(colision.p2)].xyz)
    choques += [
        dict(
            t = sim.t,
            p1 = int(colision.p1),
            p2 = int(colision.p2),
            target = xyz_1,
            position = xyz_2,
        )
    ]
    return 2

  # Modifica simulacion
  Np = sim.N
  sim.integrator = 'ias15'
  sim.dt = 100
  #sim.collision = 'direct'
  sim.collision = 'line'
  #sim.collision_resolve = 'merge'
  sim.collision_resolve = que_hacer_en_colision
  sim.heartbeat = heartbeat

  # sim.collision_resolve = 'halt'

  for n,elementos in enumerate(muestra_elementos):
    # Elementos: ['e', 'q', 'tp', 'node', 'peri', 'i']
    e = elementos[0]
    q = elementos[1]*au
    jd_p = elementos[2]
    node = elementos[3]*deg
    peri = elementos[4]*deg
    i = elementos[5]*deg

    # Elementos derivados
    a = q/(1-e)
    n = np.sqrt(mu/a**3)
    M_0 = np.mod(n*(jd_0 - jd_p)*dia,2*np.pi)

    # A partir de los elementos calcula la posición inicial del asteroide
    X_ast_0 = spy.conics([q,e,i,node,peri,M_0,0,mu],0)
    r_ast_0 = X_ast_0[:3]
    v_ast_0 = X_ast_0[3:]

    sim.add(m=0,
            x=r_ast_0[0],y=r_ast_0[1],z=r_ast_0[2],
            vx=v_ast_0[0],vy=v_ast_0[1],vz=v_ast_0[2])

  print(f"Comienzan: {sim.N-Np} asteroides")

  # Integra
  sim.integrate(tiempo_seg)
  sim.move_to_hel()
  print(f"Sobreviven: {sim.N-Np} asteroides")

  # Posición de la Tierra
  r_tierra = np.array(sim.particles[3].xyz)

  asteroides = []
  for n in range(-1,-(sim.N-Np)-1,-1):
    r_ast = np.array(sim.particles[n].xyz)
    d = np.linalg.norm(r_ast - r_tierra)
    asteroides += [
        dict(
            x_ast=r_ast[0]/au,
            y_ast=r_ast[1]/au,
            z_ast=r_ast[2]/au,
            d_ast_tie=d/au,
        )
    ]
  asteroides = pd.DataFrame(asteroides)

  # Devuelve posición del asteroide y de la Tierra
  return sim, r_tierra/au, asteroides, choques

In [ ]:
np.random.seed = 3
it = 0
paso = time()
muestra_elementos = np.random.multivariate_normal(promedios,covarianza,3000)
sim, r_tierra, asteroides, choques = integra_sistema_solar_asteroide_2(muestra_elementos,jd_imp)

Integrando hasta: 250265375.99998713
Comienzan: 3000 asteroides
Tiempo: 0.00e+00 (it = 0/50, tiempo = 0.14 seg, faltan = 6.90 seg)
Tiempo: 2.03e-02 (it = 1/50, tiempo = 89.53 seg, faltan = 4297.60 seg)
Tiempo: 4.05e-02 (it = 2/50, tiempo = 93.68 seg, faltan = 4402.95 seg)
Tiempo: 6.03e-02 (it = 3/50, tiempo = 83.51 seg, faltan = 3841.48 seg)
Tiempo: 8.02e-02 (it = 4/50, tiempo = 84.90 seg, faltan = 3820.63 seg)
Tiempo: 1.00e-01 (it = 5/50, tiempo = 80.50 seg, faltan = 3542.21 seg)
Tiempo: 1.20e-01 (it = 6/50, tiempo = 78.98 seg, faltan = 3396.20 seg)
Tiempo: 1.40e-01 (it = 7/50, tiempo = 80.83 seg, faltan = 3394.94 seg)
Tiempo: 1.60e-01 (it = 8/50, tiempo = 85.02 seg, faltan = 3485.78 seg)
Tiempo: 1.80e-01 (it = 9/50, tiempo = 84.96 seg, faltan = 3398.27 seg)
Tiempo: 2.00e-01 (it = 10/50, tiempo = 85.22 seg, faltan = 3323.72 seg)
Tiempo: 2.20e-01 (it = 11/50, tiempo = 81.97 seg, faltan = 3114.78 seg)
Tiempo: 2.40e-01 (it = 12/50, tiempo = 80.71 seg, faltan = 2986.45 seg)
Tiempo: 2.60e-

Se han truncado las últimas 5000 líneas del flujo de salida.
    self, other = self._align_for_op(other, axis, flex=True, level=None)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/frame.py", line 8187, in _align_for_op
    right = to_series(right)
            ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/frame.py", line 8133, in to_series
    raise ValueError(
ValueError: Unable to coerce to Series, length must be 0: given 1
Exception ignored on calling ctypes callback function: <function integra_sistema_solar_asteroide_2.<locals>.que_hacer_en_colision at 0x7b042f6bc9a0>
Traceback (most recent call last):
  File "<ipython-input-501-58734245c426>", line 31, in que_hacer_en_colision
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/generic.py", line 12719, in __iadd__
    return self._inplace_method(other, type(self).__add__)  # type: ignore[operator]
          

Tiempo: 1.00e+00 (it = 50/50, tiempo = 693.83 seg, faltan = -693.83 seg)
Sobreviven: 3000 asteroides


In [ ]:
choques = pd.DataFrame(choques)
distancias = np.array([np.linalg.norm(t-i) for t,i in zip(choques.target,choques.position)])/6371e3
len(choques)

AttributeError: 'DataFrame' object has no attribute 'target'

In [ ]:
asteroides.to_csv('asteroides-2024yr4.csv')
choques.to_csv('impacto-asteroide-2024yr4.csv')
np.savetxt('posicion-tierra.csv',r_tierra)

In [ ]:
R = 6371e3/au
fig = go.Figure(
    data = [
        go.Scatter(
          x=(asteroides.x_ast-r_tierra[0])/R,
          y=(asteroides.y_ast-r_tierra[1])/R,
          name='Asteroid',
          mode='markers',
          marker=dict(size=3)
        ),
    ],
    layout = dict(
      width = 600,
      height = 600,
      template = 'plotly_dark',
      yaxis=dict(scaleanchor="x", scaleratio=1)
    )
)

fig.add_shape(
    type='circle',
    xref='x', yref='y',
    x0=-1, y0=-1,
    x1=+1, y1=+1,
    line_color='Green',
    name='Tierra'
)

fig.show()